In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
root = Path().resolve().parent
competition_data_path = root / "data/competition/competition_data/"
results_root = root / "predictions/competition_baselines"

In [3]:
def save_df(test_df, data_path, competition_data_path, results_save_path):
    save_filepath = results_save_path / data_path.relative_to(competition_data_path)
    save_filepath.parent.mkdir(parents=True, exist_ok=True)
    test_df.round(3).to_csv(save_filepath, index=False)

Only fold-wise model was used for the paper describing results

# Average predictions

In [4]:
def get_subjects_paths(data_path):
    subjects_paths = dict()
    for file_path in data_path.glob("**/train/annotations/*.csv"):
        subject_num = file_path.stem.split("_")[1]
        subjects_paths.setdefault(subject_num, list())
        subjects_paths[subject_num].append(file_path)
    return subjects_paths

def get_videos_paths(data_path):
    videos_paths = dict()
    for file_path in data_path.glob("**/train/annotations/*.csv"):
        vid_num = file_path.stem.split("_")[3]
        videos_paths.setdefault(vid_num, list())
        videos_paths[vid_num].append(file_path)
    return videos_paths

def iter_subjects_videos_paths(scenario_path, search_pattern="**/train/annotations/*.csv"):
    for data_path in scenario_path.glob(pattern=search_pattern):
        yield data_path

def iter_train_subjects_and_get_paths(scenario_path):
    subjects_paths = get_subjects_paths(scenario_path)
    for subject_num, subjects_paths_list in subjects_paths.items():
        yield subject_num, subjects_paths_list

def iter_train_videos_and_get_paths(scenario_path):
    videos_paths = get_videos_paths(scenario_path)
    for video_num, videos_paths_list in videos_paths.items():
        yield video_num, videos_paths_list

## Scenario 1

In [5]:
scenario1_data_path = competition_data_path / "scenario_1"

### Fold-wise model

In [6]:
# prepare paths
model_name = "fold-wise_mean"
results_save_path = results_root / model_name / "results"

# compute baseline
arousal_results = list()
valence_results = list()
for train_path in iter_subjects_videos_paths(scenario1_data_path):
    mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
    arousal_results.append(mean["arousal"])
    valence_results.append(mean["valence"])
overall_arousal_mean = np.mean(arousal_results)
overall_valence_mean = np.mean(valence_results)

# save predictions
for test_path in iter_subjects_videos_paths(scenario1_data_path, search_pattern="**/test/annotations/*.csv"):
    test_df = pd.read_csv(test_path)
    test_df["arousal"] = overall_arousal_mean
    test_df["valence"] = overall_valence_mean
    save_df(test_df, test_path, competition_data_path, results_save_path)

### Subject-wise model

In [7]:
# prepare paths
model_name = "subject-wise_mean"
results_save_path = results_root / model_name / "results"

# compute baseline
for subject_num, paths_list in iter_train_subjects_and_get_paths(scenario1_data_path):
    subject_arousal_results = list()
    subject_valence_results = list()
    for train_path in paths_list:
        mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
        subject_arousal_results.append(mean["arousal"])
        subject_valence_results.append(mean["valence"])
    subject_arousal_mean = np.mean(subject_arousal_results)
    subject_valence_mean = np.mean(subject_valence_results)
    # save predictions
    for test_path in iter_subjects_videos_paths(scenario1_data_path, search_pattern=f"**/test/annotations/sub_{subject_num}*.csv"):
        test_df = pd.read_csv(test_path)
        test_df["arousal"] = subject_arousal_mean
        test_df["valence"] = subject_valence_mean
        save_df(test_df, test_path, competition_data_path, results_save_path)

### Video-wise model

In [8]:
# prepare paths
model_name = "video-wise_mean"
results_save_path = results_root / model_name / "results"

# compute baseline
for video_num, paths_list in iter_train_videos_and_get_paths(scenario1_data_path):
    video_arousal_results = list()
    video_valence_results = list()
    for train_path in paths_list:
        mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
        video_arousal_results.append(mean["arousal"])
        video_valence_results.append(mean["valence"])
    video_arousal_mean = np.mean(video_arousal_results)
    video_valence_mean = np.mean(video_valence_results)
    # save predictions
    for test_path in iter_subjects_videos_paths(scenario1_data_path, search_pattern=f"**/test/annotations/*vid_{video_num}.csv"):
        test_df = pd.read_csv(test_path)
        test_df["arousal"] = video_arousal_mean
        test_df["valence"] = video_valence_mean
        save_df(test_df, test_path, competition_data_path, results_save_path)

### Subject-video-wise model

In [9]:
# prepare paths
model_name = "subvid-wise_mean"
results_save_path = results_root / model_name / "results"

# compute baseline
arousal_results = list()
valence_results = list()
for train_path in iter_subjects_videos_paths(scenario1_data_path):
    mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
    mean_arousal, mean_valence = mean["arousal"], mean["valence"]
    # save predictions
    test_path = Path(str(train_path).replace("train", "test"))
    test_df = pd.read_csv(test_path)
    test_df["arousal"] = mean_arousal
    test_df["valence"] = mean_valence
    save_df(test_df, test_path, competition_data_path, results_save_path)

## Scenario 2

In [10]:
scenario2_data_path = competition_data_path / "scenario_2"

### Fold-wise model

In [11]:
# prepare paths
model_name = "fold-wise_mean"
results_save_path = results_root / model_name / "results"

for fold_num in range(5):
    fold_data_path = scenario2_data_path / f"fold_{fold_num}"
    # compute baseline
    arousal_results = list()
    valence_results = list()
    for train_path in iter_subjects_videos_paths(fold_data_path):
        mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
        arousal_results.append(mean["arousal"])
        valence_results.append(mean["valence"])
    overall_arousal_mean = np.mean(arousal_results)
    overall_valence_mean = np.mean(valence_results)

    # save predictions
    for test_path in iter_subjects_videos_paths(fold_data_path, search_pattern="**/test/annotations/*.csv"):
        test_df = pd.read_csv(test_path)
        test_df["arousal"] = overall_arousal_mean
        test_df["valence"] = overall_valence_mean
        save_df(test_df, test_path, competition_data_path, results_save_path)

### Video-wise model

In [12]:
# prepare paths
model_name = "video-wise_mean"
results_save_path = results_root / model_name / "results"

for fold_num in range(5):
    fold_data_path = scenario2_data_path / f"fold_{fold_num}"
    # compute baseline
    for video_num, paths_list in iter_train_videos_and_get_paths(fold_data_path):
        video_arousal_results = list()
        video_valence_results = list()
        for train_path in paths_list:
            mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
            video_arousal_results.append(mean["arousal"])
            video_valence_results.append(mean["valence"])
        video_arousal_mean = np.mean(video_arousal_results)
        video_valence_mean = np.mean(video_valence_results)
        # save predictions
        for test_path in iter_subjects_videos_paths(fold_data_path, search_pattern=f"**/test/annotations/*vid_{video_num}.csv"):
            test_df = pd.read_csv(test_path)
            test_df["arousal"] = video_arousal_mean
            test_df["valence"] = video_valence_mean
            save_df(test_df, test_path, competition_data_path, results_save_path)

## Scenario 3

In [13]:
scenario3_data_path = competition_data_path / "scenario_3"

### Fold-wise model

In [14]:
# prepare paths
model_name = "fold-wise_mean"
results_save_path = results_root / model_name / "results"

for fold_num in range(4):
    fold_data_path = scenario3_data_path / f"fold_{fold_num}"
    # compute baseline
    arousal_results = list()
    valence_results = list()
    for train_path in iter_subjects_videos_paths(fold_data_path):
        mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
        arousal_results.append(mean["arousal"])
        valence_results.append(mean["valence"])
    overall_arousal_mean = np.mean(arousal_results)
    overall_valence_mean = np.mean(valence_results)

    # save predictions
    for test_path in iter_subjects_videos_paths(fold_data_path, search_pattern="**/test/annotations/*.csv"):
        test_df = pd.read_csv(test_path)
        test_df["arousal"] = overall_arousal_mean
        test_df["valence"] = overall_valence_mean
        save_df(test_df, test_path, competition_data_path, results_save_path)

### Subject-wise model

In [15]:
# prepare paths
model_name = "subject-wise_mean"
results_save_path = results_root / model_name / "results"

# compute baseline
for fold_num in range(4):
    fold_data_path = scenario3_data_path / f"fold_{fold_num}"
    for subject_num, paths_list in iter_train_subjects_and_get_paths(fold_data_path):
        subject_arousal_results = list()
        subject_valence_results = list()
        for train_path in paths_list:
            mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
            subject_arousal_results.append(mean["arousal"])
            subject_valence_results.append(mean["valence"])
        subject_arousal_mean = np.mean(subject_arousal_results)
        subject_valence_mean = np.mean(subject_valence_results)
        # save predictions
        for test_path in iter_subjects_videos_paths(fold_data_path, search_pattern=f"**/test/annotations/sub_{subject_num}*.csv"):
            test_df = pd.read_csv(test_path)
            test_df["arousal"] = subject_arousal_mean
            test_df["valence"] = subject_valence_mean
            save_df(test_df, test_path, competition_data_path, results_save_path)

## Scenario 4

In [16]:
scenario4_data_path = competition_data_path / "scenario_4"

### Fold-wise model

In [17]:
# prepare paths
model_name = "fold-wise_mean"
results_save_path = results_root / model_name / "results"

for fold_num in range(2):
    fold_data_path = scenario4_data_path / f"fold_{fold_num}"
    # compute baseline
    arousal_results = list()
    valence_results = list()
    for train_path in iter_subjects_videos_paths(fold_data_path):
        mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
        arousal_results.append(mean["arousal"])
        valence_results.append(mean["valence"])
    overall_arousal_mean = np.mean(arousal_results)
    overall_valence_mean = np.mean(valence_results)

    # save predictions
    for test_path in iter_subjects_videos_paths(fold_data_path, search_pattern="**/test/annotations/*.csv"):
        test_df = pd.read_csv(test_path)
        test_df["arousal"] = overall_arousal_mean
        test_df["valence"] = overall_valence_mean
        save_df(test_df, test_path, competition_data_path, results_save_path)

### Subject-wise model

In [18]:
# prepare paths
model_name = "subject-wise_mean"
results_save_path = results_root / model_name / "results"

# compute baseline
for fold_num in range(2):
    fold_data_path = scenario4_data_path / f"fold_{fold_num}"
    for subject_num, paths_list in iter_train_subjects_and_get_paths(fold_data_path):
        subject_arousal_results = list()
        subject_valence_results = list()
        for train_path in paths_list:
            mean = pd.read_csv(train_path)[["arousal", "valence"]].mean()
            subject_arousal_results.append(mean["arousal"])
            subject_valence_results.append(mean["valence"])
        subject_arousal_mean = np.mean(subject_arousal_results)
        subject_valence_mean = np.mean(subject_valence_results)
        # save predictions
        for test_path in iter_subjects_videos_paths(fold_data_path, search_pattern=f"**/test/annotations/sub_{subject_num}*.csv"):
            test_df = pd.read_csv(test_path)
            test_df["arousal"] = subject_arousal_mean
            test_df["valence"] = subject_valence_mean
            save_df(test_df, test_path, competition_data_path, results_save_path)